# Interactive Visualization of `EHRData` with Vitessce

This tutorial demonstrates how to create interactive visualizations of EHRData objects using [Vitessce](https://vitessce.io/).

Vitessce provides linked, coordinated views that allow you to explore clinical data interactively in a web browser or Jupyter notebook.


## Load Data


In [1]:
import ehrdata as ed
from pathlib import Path

In [ ]:
# Load a subset of PhysioNet 2012 data
# TODO: tabular? I think better longitudinal.
# and then explain that can take slice.
edata = ed.dt.physionet2012(
    data_path=Path("/Users/eljas.roellin/Documents/ehrapy_workspace/ehrdata/ehrapy_data/physionet2012"),
    layer="tem_data",
    # n_subsamples=500, subsample_seed=42
)
edata

View of EHRData object with n_obs × n_vars × n_t = 11988 × 37 × 48
    obs: 'set', 'Age', 'Gender', 'Height', 'ICUType', 'SAPS-I', 'SOFA', 'Length_of_stay', 'Survival', 'In-hospital_death'
    var: 'Parameter'
    tem: '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47'
    layers: 'tem_data'
    shape of .tem_data: (11988, 37, 48)

## Save to Zarr Format

Vitessce reads data from the Zarr format. Let's save our data:


In [3]:
!pip show zarr

Name: zarr
Version: 3.1.5
Summary: An implementation of chunked, compressed, N-dimensional arrays for Python
Home-page: https://github.com/zarr-developers/zarr-python
Author: 
Author-email: Alistair Miles <alimanfoo@googlemail.com>
License-Expression: MIT
Location: /Users/eljas.roellin/miniconda3/envs/ehrdata_ehrapy_env_19sep/lib/python3.13/site-packages
Requires: donfig, google-crc32c, numcodecs, numpy, packaging, typing-extensions
Required-by: anndata, ehrdata, kerchunk, ome-zarr, vitessce


In [4]:
# Save to Zarr
zarr_path = Path("physionet2012_subset.zarr")
ed.io.write_zarr(edata, zarr_path)
print(f"Data saved to {zarr_path}")

Data saved to physionet2012_subset.zarr


In [ ]:
import anndata as ad

# TODO: hide this from user.
ad.AnnData(edata).write_zarr(zarr_path)

## Generate Vitessce Configuration

The `ehrdata.integrations.vitessce.gen_config()` helper function creates a Vitessce configuration tailored for your EHRData object.

You can specify which patient attributes to visualize using the `obs_sets` parameter:


In [8]:
# Generate Vitessce configuration
vc = ed.integrations.vitessce.gen_config(
    path=zarr_path,
    name="PhysioNet 2012 ICU Data",
    obs_sets={
        "obs/Gender": "Gender",
        "obs/ICUType": "ICU Type",
        "obs/In-hospital_death": "Mortality",
        "obs/set": "Hospital Set",
    },
    obs_embeddings={},
)

## Display the Interactive Visualization

Now display the interactive widget! You can:
- **Select patient groups** by gender, ICU type, or mortality outcome
- **Explore feature distributions** across different patient groups
- **View heatmaps** of clinical variables
- **All views are linked** - selections in one view update all others


In [ ]:
# Display the widget
# TODO: make a nicer one
vc.widget()

VitessceWidget(uid='4538')

## Summary

In this tutorial, we learned:

- ✅ How to save EHRData to Zarr format with `.write_zarr()`
- ✅ How to use `ed.integrations.vitessce.gen_config()` to create interactive visualizations
- ✅ How to specify which patient attributes to visualize with `obs_sets`
- ✅ How to display Vitessce widgets in Jupyter notebooks

The `gen_config()` helper handles the complexity of setting up Vitessce, allowing you to focus on exploring your clinical data!

## Where to go next

### 🏥 Working with OMOP Data
- **[OMOP Common Data Model](omop_intro)** - Load standardized healthcare data from OMOP databases and construct EHRData objects step-by-step.
